# Notebook for a space-time discretization

In this notebook we are treating following problem 
$$
u_t - \Delta u + \mathbf{w} \cdot \nabla u = f \quad \text{in } \Omega  = [0,1] \times [0,T].
$$
with the initial/boundary conditions
$$
u(x,0) = \sin(\pi x), \quad u(0,t) = u(1,t)=0.
$$
The idea of the method is to replace the testfunction v with the elementwisefunction  $v +\gamma_t \cdot w \cdot v_x$

The weakformulation of the problem in the SUPG setting is

Find a function $u \in H^1_0(\Omega)$ , such that  for all  $v \in H^1_0(\Omega) $:

$$
\int_{\Omega} (u_t) v \,dx \,dt
+ \int_{\Omega} \nabla u \cdot \nabla v \,dx \,dt
+ \int_{\Omega} (\mathbf{w} \cdot \nabla u) v \,dx \,dt
+ \sum_{T \in T_h} \int_{T} \gamma_T \cdot (u_t- u_{xx} + \mathbf{w} \cdot \nabla u -f) v_x \cdot\mathbf{w} \,dx \,dt
= \int_{\Omega} f v \,dx \,dt

$$

For $w = 0$ (the heatequation), we have for the chosen initial/boundary conditions following analytical solution 

$u(x,t)= \sin(\pi x) e^{-t \pi^2}$

In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

T = 2 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [5]:
order = 6
fes = H1(mesh, order=order, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 5 # the wind speed
h = specialcf.mesh_size

gamma_0 = 1
gamma_T = gamma_0 * h**2 / (w * order**2)

ux = CoefficientFunction((grad(u)[0],0))
ut = CoefficientFunction((0,grad(u)[1]))
B = CoefficientFunction((w,1))
uxx = CoefficientFunction((u.Operator("hesse")[0,0],0))

a = BilinearForm(fes, symmetric=False)
a += ux*grad(v)*dx + (B*grad(u))*v*dx
a += gamma_T * (-uxx + w * ux + ut)*(CoefficientFunction((w*grad(v)[0],grad(v)[1]))) * dx
a.Assemble()

f = LinearForm(fes)
f += 0*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(exp(w*x/2)*sin(pi*x),BND)  # initial condition
Draw(gfu)

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

l2error = sqrt(Integrate((gfu-(sin(pi*x)*exp(-(pi**2+w**2/4)*y)*exp(w*x/2)))**2, mesh))
print(l2error)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.00010613076694257403
